In [1]:
import numpy as np
import cv2
import glob, os
import matplotlib.pyplot as plt
import PyQt5
%matplotlib inline

In [2]:
from ImageProcess import ImageProcess
from LaneFinding import Lanes

import pickle
if not os.path.exists("./data/cali_pickle.p"):
    print("Run camera_calibraton.ipynb first.")
else:    
    cali_pickle = pickle.load( open( "./data/cali_pickle.p", "rb" ))
    
ImageProcess = ImageProcess(cali_pickle)
Lanes = Lanes(cali_pickle)

In [3]:
def pipeline(img):
    undistorted = ImageProcess.undistort(img)
    combined_binary_color, combined_binary = ImageProcess.combine_thresh(undistorted)
    warped = ImageProcess.perspective(combined_binary)
    if (Lanes.left_fit_pre is None) or (Lanes.right_fit_pre is None) or (Lanes.lane_width < 2.5):
        lines_img = Lanes.find_lines_initial(warped)
    else:
        lines_img = Lanes.find_lines(warped)
        
    left_curv, right_curv = Lanes.curvature()
    offset, lane_width = Lanes.lane_offset()
    lane_img = Lanes.draw_lane(img, warped)
    
    output_image = np.ones((img.shape[0] + 304, img.shape[1], 3))*255
    output_image[0:img.shape[0], 0:img.shape[1]] = lane_img
    
    img1 = cv2.resize(combined_binary_color, (420,300))
    img2 = cv2.resize(warped, (420,300))
    img3 = cv2.resize(lines_img, (420,300))

    output_image[-306:-6, 0:420] = img1*255
    output_image[-306:-6, 430:850,0] = img2*255
    output_image[-306:-6, 430:850,1] = img2*255
    output_image[-306:-6, 430:850,2] = img2*255
    output_image[-306:-6, 860:1280] = img3
    
    font_type = cv2.FONT_HERSHEY_DUPLEX 
    cv2.putText(output_image, "Color Binary", (30, 750), font_type, 0.8, (255,255,255), 2)
    cv2.putText(output_image, "Warped Image", (460, 750), font_type, 0.8, (255,255,255), 2)
    cv2.putText(output_image, "Lines Fitting", (890, 750), font_type, 0.8, (255,255,255), 2)
    
    text_curve_l = "Left curvature: {:.2f} m".format(left_curv)
    text_curve_r = "Right curvature: {:.2f} m".format(right_curv)
    text_offset = "Lane offset: {:.2f} m".format(offset)
    
    cv2.putText(output_image, text_curve_l, (40, 40), font_type, 0.8, (38,153,38), 2)
    cv2.putText(output_image, text_curve_r, (40, 70), font_type, 0.8, (38,153,38), 2)
    cv2.putText(output_image, text_offset, (40, 100), font_type, 0.8, (38,153,38), 2)
    
    return output_image

In [4]:
from moviepy.editor import VideoFileClip
video_output = 'project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4");
white_clip = clip1.fl_image(pipeline) 
%time white_clip.write_videofile(video_output, audio = False);

ModuleNotFoundError: No module named 'moviepy'

In [ ]:
from IPython.display import HTML
HTML("""
<video width="800" height="600" controls>
  <source src="{0}">
</video>
""".format(video_output))

In [5]:
video_output2 = 'challenge_video_output.mp4'
clip2 = VideoFileClip("challenge_video.mp4");
white_clip = clip2.fl_image(pipeline) 
%time white_clip.write_videofile(video_output2, audio = False);

NameError: name 'VideoFileClip' is not defined

In [6]:
from IPython.display import HTML
HTML("""
<video width="640" height="360" controls>
  <source src="{0}">
</video>
""".format(video_output2))

In [7]:
video_output3 = 'harder_challenge_video_output.mp4'
clip3 = VideoFileClip("harder_challenge_video.mp4");
white_clip = clip3.fl_image(pipeline) 
%time white_clip.write_videofile(video_output3, audio = False);

NameError: name 'VideoFileClip' is not defined

In [8]:
from IPython.display import HTML
HTML("""
<video width="640" height="360" controls>
  <source src="{0}">
</video>
""".format(video_output3))